In [ ]:
"""Final figures for paper."""

# Imports
import os
import re
from glob import glob

import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

DOT_LW = 1.0

In [ ]:
# Helper functions
def get_mean(df, threshold=0):
    """Function to get the mean at a given threshold."""
    if isinstance(threshold, str):
        threshold = float(threshold)
    return df.set_index("threshold").loc[threshold]["nav_mean"]


def get_aborted(df, threshold=0):
    """Function to get the mean at a given threshold."""
    if isinstance(threshold, str):
        threshold = float(threshold)
    aborted = df.set_index("threshold").loc[threshold]["aborted_mean"]
    if np.isnan(aborted):
        return 1.0
    return aborted

In [ ]:
# Set fonts
# font = {
#     "family": "serif",
#     "serif": ["Computer Modern Roman"],
#     "weight": "normal",
#     "size": 10,
# }

OUT_PATH = "outs_final_nee"
os.makedirs(OUT_PATH, exist_ok=True)

DATA_PATH = "gp_maps_nav_data/"


ONE_COL_MM = 890

nee_fs = 7
font = {
    "family": "sans-serif",
    "sans-serif": ["Helvetica"],
    "weight": "normal",
    "size": nee_fs,
}
plt.matplotlib.rc("font", **font)
# plt.matplotlib.rcParams['text.usetex'] = True

figsize = (ONE_COL_MM / 250, 6/5 * ONE_COL_MM / 250)

# Constants
OUTS_FINAL = OUT_PATH
DPI = 150
COMP_ONLY = 0


# Make outpaths
os.makedirs(OUTS_FINAL, exist_ok=True)

# Set-up matplotlib style
plt.matplotlib.rc("font", **font)

SMALL_RCPARAMS = {
    "ytick.major.size": 2,
    "ytick.minor.size": 1,
    "xtick.major.size": 2,
    "xtick.minor.size": 1,
}

In [ ]:
# Aggregate figure: n=100, "most common" fitness measure
def plot_main_comparison(fitness_type, figsize=None):
    paths = [
        f"{DATA_PATH}/data/raw_output/20220213_outs_rna_mono_rdo/random/l20/n100/analysis/fitness_type-mono/diff_thresh-0.00/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        f"{DATA_PATH}/data/raw_output/20220220_outs_rna_mul_001/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        f"{DATA_PATH}/data/raw_output/20220220_outs_rna_mul_1/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
    ]

    # fig, axs = plt.subplots(3, 3, sharey=True, sharex=True)
    fig, axs = plt.subplots(3, 3, sharey=True, sharex=True, figsize=figsize)
    bins = np.arange(0, 1.01, 0.1)
    alpha = 0.5
    ylim = (0, 50)
    # title_fs = "medium"
    title_fs = nee_fs
    lengths = ["l20", "l30", "l40"]
    for row, length in enumerate(lengths):
        for i, path in enumerate(paths):
            ax = axs[row, i]

            # Random
            # Get hist
            df = pd.read_csv(path.replace("l20", length))
            if not df["0.0"].isna().all():
                ax.hist(df["0.0"], bins=bins, alpha=alpha, color="blue", label="Random")

            # Get mean for threshold 0
            df = pd.read_csv(
                path.replace("l20", length).replace("target", "source-target")
            )
            ax.vlines(
                get_mean(df, threshold=0.0),
                *ylim,
                color="blue",
                linestyles="--",
                label="_nolegend_",
            )

            # Get Aborted
            ax2 = ax.twinx()
            ax2.hlines(
                get_aborted(df, threshold=0.0),
                0,
                1,
                color="blue",
                linestyles=":",
                label="aborted\n(random)",
            )
            ax2.set_ylim(0, 1)

            # Hamming
            # Get hist
            df = pd.read_csv(path.replace("l20", length).replace("random", "hamming"))
            if not df["0.0"].isna().all():
                ax.hist(
                    df["0.0"], bins=bins, alpha=alpha, color="orange", label="Hamming"
                )

            df = pd.read_csv(
                path.replace("l20", length)
                .replace("random", "hamming")
                .replace("target", "source-target")
            )
            ax.vlines(
                get_mean(df), *ylim, color="orange", linestyles="--", label="_nolegend_"
            )

            ax2.hlines(
                get_aborted(df),
                0,
                1,
                color="orange",
                linestyles=":",
                label="aborted\n(hamming)",
            )
            ax2.set_ylim(0, 1)

            ax.set_ylim(*ylim)

            if i != len(paths) - 1:
                ax2.axes.get_yaxis().set_ticklabels([])
            if i == len(paths) - 1 and row == 1:
                ax2.set_ylabel(r"Aborted, $\alpha$")

    axs[0, 0].set_title("Monomorphic\n" r"$N \mu L \ll 1$", fontsize=title_fs)
    axs[0, 1].set_title("Intermediate\n" r"$N \mu L = 1$", fontsize=title_fs)
    axs[0, 2].set_title("Polymorphic\n" r"$N \mu L = 100$", fontsize=title_fs)

    axs[-1, 1].set_xlabel(
        r"Phenotypic evolutionary navigability, $\left<\psi_p^\mathrm{evo}\right>$"
    )

    axs[1, 0].set_ylabel(r"Count")

    for i, length in enumerate(lengths):
        axs[i, 0].text(
            -0.35,
            1.05,
            f"L={length[-2:]}",
            transform=axs[i, 0].transAxes,
            ha="left",
            weight="bold",
        )

    # for j in range(2, 3):
    #     axs[0, j].legend(
    #         loc="upper left",
    #         bbox_to_anchor=(1.15, 1.0),
    #         prop={"size": "x-small"},
    #     )

    axs[0, 0].legend(
        loc="lower left",
        bbox_to_anchor=(0.0, 1.32),
        prop={"size": "medium"},
        ncol=2,
    )

    fname = f"random_vs_hamming_n100_{fitness_type}"
    plt.savefig(
        os.path.join(OUTS_FINAL, f"{fname}.pdf"), transparent=True, bbox_inches="tight"
    )
    plt.savefig(
        os.path.join(OUTS_FINAL, f"{fname}.png"),
        transparent=True,
        bbox_inches="tight",
        dpi=DPI,
    )
    plt.show()

In [ ]:
# Aggregate figure: n=100, "most common" fitness measure
def plot_main_comparison_4(fitness_type, COMP_ONLY=0):
    paths = [
        f"{DATA_PATH}/data/raw_output/20220213_outs_rna_mono_rdo/random/l20/n100/analysis/fitness_type-mono/diff_thresh-0.00/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        f"{DATA_PATH}/data/raw_output/20220220_outs_rna_mul_001/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        f"{DATA_PATH}/data/raw_output/20220220_outs_rna_mul_01/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        f"{DATA_PATH}/data/raw_output/20220220_outs_rna_mul_1/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
    ]

    fig, axs = plt.subplots(3, 4, sharey=True, sharex=True)
    bins = np.arange(0, 1.01, 0.1)
    alpha = 0.5
    ylim = (0, 50)
    # title_fs = "medium"
    title_fs = nee_fs
    lengths = ["l20", "l30", "l40"]
    for row, length in enumerate(lengths):
        for i, path in enumerate(paths):
            ax = axs[row, i]

            # Random
            # Get hist
            df = pd.read_csv(path.replace("l20", length))
            if not df["0.0"].isna().all():
                ax.hist(df["0.0"], bins=bins, alpha=alpha, color="blue", label="Random")

            # Get mean for threshold 0
            df = pd.read_csv(
                path.replace("l20", length).replace("target", "source-target")
            )
            ax.vlines(
                get_mean(df, threshold=0.0),
                *ylim,
                color="blue",
                linestyles="--",
                label="_nolegend_",
            )

            # Get Aborted
            ax2 = ax.twinx()
            ax2.hlines(
                get_aborted(df, threshold=0.0),
                0,
                1,
                color="blue",
                linestyles=":",
                label="aborted\n(random)",
            )
            ax2.set_ylim(0, 1)

            # Hamming
            # Get hist
            df = pd.read_csv(path.replace("l20", length).replace("random", "hamming"))
            if not df["0.0"].isna().all():
                ax.hist(
                    df["0.0"], bins=bins, alpha=alpha, color="orange", label="Hamming"
                )

            df = pd.read_csv(
                path.replace("l20", length)
                .replace("random", "hamming")
                .replace("target", "source-target")
            )
            ax.vlines(
                get_mean(df), *ylim, color="orange", linestyles="--", label="_nolegend_"
            )

            ax2.hlines(
                get_aborted(df),
                0,
                1,
                color="orange",
                linestyles=":",
                label="aborted\n(hamming)",
            )
            ax2.set_ylim(0, 1)

            ax.set_ylim(*ylim)

            if i != len(paths) - 1:
                ax2.axes.get_yaxis().set_ticklabels([])
            if i == len(paths) - 1 and row == 1:
                ax2.set_ylabel(r"Aborted, $\alpha$")

    # axs[0, 0].set_title("Monomorphic\n" r"$N \mu L \ll 1$", fontsize=title_fs)
    # axs[0, 1].set_title("Intermediate\n" r"$N \mu L = 1$", fontsize=title_fs)
    # axs[0, 2].set_title("Polymorphic\n" r"$N \mu L = 10$", fontsize=title_fs)
    # axs[0, 3].set_title("Polymorphic\n" r"$N \mu L = 100$", fontsize=title_fs)

    axs[0, 0].set_title(r"$N \mu L \ll 1$", fontsize=title_fs)
    axs[0, 1].set_title(r"$N \mu L = 1$", fontsize=title_fs)
    axs[0, 2].set_title(r"$N \mu L = 10$", fontsize=title_fs)
    axs[0, 3].set_title(r"$N \mu L = 100$", fontsize=title_fs)

    # Put a common label for the x-axis using the fig coordinate system
    central = 0.5 * (
        axs[-1, 1].get_position().bounds[0]
        + axs[-1, 1].get_position().bounds[1]
        + axs[-1, 2].get_position().bounds[0]
    )
    fig.text(
        central,
        0.03,
        r"Phenotypic evolutionary navigability, $\left<\psi_p^\mathrm{evo}\right>$",
        ha="center",
        va="bottom",
    )
    axs[1, 0].set_ylabel(r"Count")

    for i, length in enumerate(lengths):
        axs[i, 0].text(
            -0.5,
            1.05,
            f"L={length[-2:]}",
            transform=axs[i, 0].transAxes,
            ha="left",
            weight="bold",
        )

    axs[0, 0].legend(
        loc="lower left",
        bbox_to_anchor=(0.0, 1.25),
        prop={"size": "medium"},
        # prop={"size": "x-small"},
        ncol=2,
    )

    fname = f"random_vs_hamming_4_n100_{fitness_type}_comp-only_{COMP_ONLY}"
    plt.savefig(
        os.path.join(OUTS_FINAL, f"{fname}.pdf"), transparent=True, bbox_inches="tight"
    )
    plt.savefig(
        os.path.join(OUTS_FINAL, f"{fname}.png"),
        transparent=True,
        bbox_inches="tight",
        dpi=DPI,
    )
    plt.show()


# Most common, n=100
plot_main_comparison_4("most_common")

# Max fit, n=100
plot_main_comparison_4("max_fit")

# Most common, n=100
plot_main_comparison_4("most_common", COMP_ONLY=1)

# Max fit, n=100
plot_main_comparison_4("max_fit", COMP_ONLY=1)

In [ ]:
def print_nav_and_abort(df, threshold=0.0, label="NOT PASSED"):
    """Print helper for data."""
    mean_nav = get_mean(df, threshold=threshold)
    mean_abo = get_aborted(df, threshold=threshold)
    print(f"{label:<10}: ψ={mean_nav:.3f}, α={mean_abo:.3f}")

In [ ]:
# Aggregate figure: n=100, "most common" fitness measure
def plot_pair_comparison(
    fitness_type,
    figsize=None,
    lengths=["l20", "l30", "l40", "l40"],
    fname="random_vs_hamming_momo_poly_pair",
    threshold = "0.0"
):
    paths = [
        f"{DATA_PATH}/data/raw_output/20220213_outs_rna_mono_rdo/random/l20/n100000/analysis/fitness_type-mono/diff_thresh-0.00/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        # f"../data/raw_output/20220220_outs_rna_mul_001/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        f"{DATA_PATH}/data/raw_output/20220220_outs_rna_mul_1/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
    ]

    # fig, axs = plt.subplots(3, 3, sharey=True, sharex=True)
    fig, axs = plt.subplots(len(lengths), 2, sharey=True, sharex=True, figsize=figsize)
    bins = np.arange(0, 1.01, 0.1)
    alpha = 0.5
    ylim = (0, 50)
    # title_fs = "medium"
    title_fs = nee_fs
    leg_axs = []
    for row, length in enumerate(lengths):
        for i, path in enumerate(paths):
            print(f"L={length[1:]}", f"{i}")
            ax = axs[row, i]

            if row == len(lengths) - 1:
                # nav_threshold = "0.075"
                nav_threshold = threshold
            else:
                nav_threshold = "0.0"

            # Random
            # Get hist
            df = pd.read_csv(path.replace("l20", length))
            if not df[nav_threshold].isna().all():
                ax.hist(
                    df[nav_threshold],
                    bins=bins,
                    alpha=alpha,
                    color="blue",
                    label="Random",
                    zorder=0,
                )

            # Get mean for threshold 0
            df = pd.read_csv(
                path.replace("l20", length).replace("target", "source-target")
            )
            ax.vlines(
                get_mean(df, threshold=nav_threshold),
                *ylim,
                color="blue",
                linestyles="--",
                label="_nolegend_",
            )
            
            # Get Aborted
            ax2 = ax.twinx()
            ax2.hlines(
                get_aborted(df, threshold=nav_threshold),
                0,
                1,
                color="blue",
                linestyles=":",
                lw=DOT_LW,
                zorder=-2,
                # label="aborted\n(random)",
                label="_nolabel_",
            )
            ax2.set_ylim(0, 1)
            print_nav_and_abort(df, float(nav_threshold), "random")
            # Hamming
            # Get hist
            df = pd.read_csv(path.replace("l20", length).replace("random", "hamming"))
            if not df[nav_threshold].isna().all():
                ax.hist(
                    df[nav_threshold],
                    bins=bins,
                    alpha=alpha,
                    color="orange",
                    label="Hamming",
                    zorder=0,
                )

            df = pd.read_csv(
                path.replace("l20", length)
                .replace("random", "hamming")
                .replace("target", "source-target")
            )
            ax.vlines(
                get_mean(df, nav_threshold),
                *ylim,
                color="orange",
                linestyles="--",
                # label="_nolegend_",
                label=r"Navigability, $\left<\psi^\mathrm{evo}\right>$",
                zorder=-1,
            )

            ax2.hlines(
                get_aborted(df, nav_threshold),
                0,
                1,
                color="orange",
                linestyles=":",
                lw=DOT_LW,
                zorder=-2,
                # label="Aborted\n(hamming)",
                label="Aborted, " r"$\alpha$",
            )
            print_nav_and_abort(df, float(nav_threshold), "Hamming")
            ax2.set_ylim(0, 1)

            ax.set_ylim(*ylim)

            # if i != len(paths) - 1:
            #     ax2.axes.get_yaxis().set_ticklabels([])
            # if i == len(paths) - 1 and row == 1:
            #     ax2.set_ylabel(r"Aborted, $\alpha$")
            # if i == 0:
                # ax.set_ylabel(r"Count")
            # if i == 1:
            # ax2.set_ylabel(r"Aborted, $\alpha$")

            if i == 0:
                ax2.set_yticklabels([])

            if row == 0 and i == 0:
                leg_axs = [ax, ax2]
                ax00_x = axs[0, 0].get_position().bounds[0]
                ax00_y = (
                    axs[0, 0].get_position().bounds[1]
                    + axs[0, 0].get_position().bounds[3]
                )
                lines_labels = [ax.get_legend_handles_labels() for ax in leg_axs]
                lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
                fig.legend(
                    lines,
                    labels,
                    loc="lower left",
                    prop={"size": "small"},
                    bbox_to_anchor=(ax00_x, ax00_y + 0.035),
                    ncol=2,
                    frameon=False,
                )

    # Count label
    pos1 = axs[1, 0].get_position()
    pos2 = axs[2, 1].get_position()
    x = pos1.bounds[0] - 0.085
    # x = pos1.bounds[0] + pos1.bounds[2] - 0.1
    y = 0.5 * (
        axs[1, 0].get_position().bounds[1]
        + axs[2, 0].get_position().bounds[1]
        + axs[2, 0].get_position().bounds[3]
    )
    fig.text(x, y, "Count", rotation=90, ha="center", va="center")

    axs[0, 0].set_yticks([0, 20, 40])
                
    # Aborted label
    pos1 = axs[1, 1].get_position()
    pos2 = axs[2, 1].get_position()
    x = pos1.bounds[0] + pos1.bounds[2] + 0.1
    y = 0.5 * (
        axs[1, 1].get_position().bounds[1]
        + axs[2, 1].get_position().bounds[1]
        + axs[2, 1].get_position().bounds[3]
    )
    fig.text(x, y, r"Aborted, $\alpha$", rotation=90, ha="center", va="center")

    axs[0, 0].set_title("Monomorphic\n" r"$N \mu L \ll 1$", fontsize=title_fs)
    axs[0, 0].set_title(r"$N \mu L \ll 1$", fontsize=title_fs)
    axs[0, 1].set_title("Polymorphic\n" r"$N \mu L = 100$", fontsize=title_fs)
    axs[0, 1].set_title("Polymorphic, " r"$N \mu L = 100$", fontsize=title_fs, pad=18)
    axs[0, 1].set_title(r"$N \mu L = 100$", fontsize=title_fs)

    # Put a common label for the x-axis using the fig coordinate system
    central = 0.5 * (
        axs[-1, 0].get_position().bounds[0]
        + axs[-1, 0].get_position().bounds[2]
        + axs[-1, 1].get_position().bounds[0]
    )
    fig.text(
        central,
        0.035,
        r"Phenotypic evolutionary navigability, $\left<\psi_p^\mathrm{evo}\right>$",
        ha="center",
        va="bottom",
    )

    # axs[1, 0].set_ylabel(r"Count")
    lsize = "small"
    
    for i, length in enumerate(lengths):
        if i != len(lengths) - 1:
            label = f"L={length[-2:]}"
        else:
            label = f"L={length[-2:]} " + r"($\Delta F$ = {0})".format(nav_threshold)
            
        # axs[i, 0].text(
        top = axs[i, 0].get_position().bounds[1] + axs[i, 0].get_position().bounds[3]
        # top = axs[i, 0].get_position().bounds[3]
        fig.text(
            # -0.16,
            # 1.15,
            1.12,
            1.05,
            label,
            transform=axs[i, 0].transAxes,
            ha="center",
            va="bottom",
        )

    plt.subplots_adjust(hspace=0.4, wspace=0.2)
    fname = f"{fname}_{fitness_type}"
    plt.savefig(
        os.path.join(OUTS_FINAL, f"{fname}.pdf"), transparent=True, bbox_inches="tight"
    )
    plt.savefig(
        os.path.join(OUTS_FINAL, f"{fname}.png"),
        transparent=True,
        bbox_inches="tight",
        dpi=DPI,
    )
    plt.show()

In [ ]:
# Pairs for mono and poly only
for nav_threshold in ["0.0", "0.025", "0.05", "0.075", "0.1"]:
    plot_pair_comparison(
        "most_common",
        # figsize=(4, 6 / 5 * 4),
        figsize = figsize,
        lengths=["l20", "l30", "l40", "l40"],
        fname=f"random_vs_hamming_momo_poly_pair_l40_{nav_threshold}",
        threshold = nav_threshold
    )

In [ ]:
for nav_threshold in ["0.0", "0.025", "0.05", "0.075", "0.1"]:
    plot_pair_comparison(
        "most_common",
        # figsize=(4, 6 / 5 * 4),
        figsize=figsize,
        lengths=["l20", "l30", "l40", "l30"],
        fname=f"random_vs_hamming_momo_poly_pair_l30_{nav_threshold}",
        threshold = nav_threshold
    )

In [ ]:
def plot_poly_comparison(
    ns=[10, 1000, 10000],
    path="20220327_outs_rna_shape_l3_mono_rdo",
    shape_fitness="hamming",
    fname="random_vs_hamming_poly",
    lengths=["l20", "l30", "l40"],
    ylim=(0, 50),
    fitness_type="most_common",
    COMP_ONLY=0,
    all_label=False,
    figsize=(3.45, 3),
):
    """Polymorphic comparisons."""
    paths = [
        f"{DATA_PATH}/data/raw_output/20220220_outs_rna_mul_001/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        f"{DATA_PATH}/data/raw_output/20220220_outs_rna_mul_01/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        f"{DATA_PATH}/data/raw_output/20220220_outs_rna_mul_1/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
    ]

    nmuls = [1, 10, 100]
    with plt.rc_context(SMALL_RCPARAMS):
        fig, axs = plt.subplots(3, 3, sharey=True, sharex=True, figsize=figsize)

        nav_threshold = "0.0"

        bins = np.arange(0, 1.01, 0.1)
        alpha = 0.5
        # ylim = (0, 50)
        # title_fs = "medium"
        title_fs = nee_fs
        # lengths = ["l20", "l30", "l40"]
        for row, length in enumerate(lengths):
            for i, path in enumerate(paths):
                mul = re.findall("mul_(\d+)/", path)[0]
                print(f"L={length[1:]}", f"NμL={nmuls[i]}")
                ax = axs[row, i]
                df = pd.read_csv(path.replace("l20", length))
                ax.hist(df["0.0"], bins=bins, alpha=alpha, color="blue", label="Random")

                df = pd.read_csv(
                    path.replace("l20", length).replace("target", "source-target")
                )

                print_nav_and_abort(df, label="random")

                ax.vlines(
                    get_mean(df),
                    *ylim,
                    color="blue",
                    linestyles="--",
                    label="_nolegend_",
                )

                ax2 = ax.twinx()
                ax2.hlines(
                    get_aborted(df),
                    0,
                    1,
                    color="blue",
                    linestyles=":",
                    lw=DOT_LW,
                    # label="aborted\n(random)",
                    label="_nolegend_",
                )
                ax2.set_ylim(0, 1)

                df = pd.read_csv(
                    path.replace("l20", length).replace("random", shape_fitness)
                )
                ax.hist(
                    df["0.0"],
                    bins=bins,
                    alpha=alpha,
                    color="orange",
                    label=shape_fitness.title(),
                )

                df = pd.read_csv(
                    path.replace("l20", length)
                    .replace("random", shape_fitness)
                    .replace("target", "source-target")
                )

                print_nav_and_abort(df, label="hamming")

                ax.vlines(
                    get_mean(df, nav_threshold),
                    *ylim,
                    color="orange",
                    linestyles="--",
                    # label="_nolegend_",
                    label=r"Navigability, $\left<\psi^\mathrm{evo}\right>$",
                    zorder=-1,
                )

                ax2.hlines(
                    get_aborted(df, nav_threshold),
                    0,
                    1,
                    color="orange",
                    linestyles=":",
                    lw=DOT_LW,
                    zorder=-2,
                    # label="Aborted\n(hamming)",
                    label="Aborted, " r"$\alpha$",
                )
                ax2.set_ylim(0, 1)

                ax.set_ylim(*ylim)

                if i != len(paths) - 1:
                    ax2.axes.get_yaxis().set_ticklabels([])
                if i == len(paths) - 1 and row == 1:
                    ax2.set_ylabel(r"Aborted, $\alpha$")

                if all_label:
                    if i == 0:
                        ax.set_ylabel(r"Count")
                    if i == len(axs[0]) - 1:
                        ax2.set_ylabel(r"Aborted, $\alpha$")
                else:
                    if i == 0 and row == len(axs)//2:
                        ax.set_ylabel(r"Count")
                    if i == len(axs[0]) - 1 and row == len(axs)//2:
                        ax2.set_ylabel(r"Aborted, $\alpha$")

                if row == 0 and i == 0:
                    leg_axs = [ax, ax2]
                    ax00_x = axs[0, 0].get_position().bounds[0]
                    ax00_y = (
                        axs[0, 0].get_position().bounds[1]
                        + axs[0, 0].get_position().bounds[3]
                    )
                    lines_labels = [ax.get_legend_handles_labels() for ax in leg_axs]
                    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
                    fig.legend(
                        lines,
                        labels,
                        loc="lower left",
                        prop={"size": "small"},
                        bbox_to_anchor=(ax00_x, ax00_y + 0.04),
                        ncol=2,
                        frameon=False,
                    )

        axs[0, 0].set_title(r"$N \mu L = 1$", fontsize=title_fs)
        axs[0, 1].set_title(r"$N \mu L = 10$", fontsize=title_fs)
        axs[0, 2].set_title(r"$N \mu L = 100$", fontsize=title_fs)

        axs[-1, 1].set_xlabel(
            r"Phenotypic evolutionary navigability, $\left<\psi_p^\mathrm{evo}\right>$"
        )

        axs[1, 0].set_ylabel(r"Count")

        for i, length in enumerate(lengths):
            axs[i, 0].text(
                -0.6,
                1.05,
                f"L={length[1:]}",
                transform=axs[i, 0].transAxes,
                ha="left",
                fontsize=nee_fs,
                weight="bold",
            )

        plt.subplots_adjust(hspace=0.3)

        plt.savefig(
            os.path.join(OUTS_FINAL, f"{fname}.pdf"),
            transparent=True,
            bbox_inches="tight",
        )
        plt.savefig(
            os.path.join(OUTS_FINAL, f"{fname}.png"),
            transparent=True,
            bbox_inches="tight",
            dpi=DPI,
        )
        plt.show()

In [ ]:
def plot_mono_comparison(
    ns=[10, 1000, 10000],
    path="20220327_outs_rna_shape_l3_mono_rdo",
    shape_fitness="lev",
    fname="random_vs_hamming_mono_shape_l3",
    lengths=["l60", "l100", "l140"],
    ylim=(0, 20),
    COMP_ONLY=0,
    all_label=False,
    figsize=(3.45, 3),
):
    """Monomorphic comparisons."""
    paths = [
        f"{DATA_PATH}/data/raw_output/{path}/random/l20/n{ns[0]}/analysis/fitness_type-mono/diff_thresh-0.00/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        f"{DATA_PATH}/data/raw_output/{path}/random/l20/n{ns[1]}/analysis/fitness_type-mono/diff_thresh-0.00/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        f"{DATA_PATH}/data/raw_output/{path}/random/l20/n{ns[2]}/analysis/fitness_type-mono/diff_thresh-0.00/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
    ]

    with plt.rc_context(SMALL_RCPARAMS):
        fig, axs = plt.subplots(
            len(lengths), 3, sharey=True, sharex=True, figsize=figsize
        )

        nav_threshold = "0.0"

        bins = np.arange(0, 1.01, 0.1)
        alpha = 0.5
        title_fs = nee_fs
        for row, length in enumerate(lengths):
            for i, path in enumerate(paths):
                print(f"L={length[1:]}", f"N={ns[i]}")

                path.replace("l20", length)
                ax = axs[row, i]
                df = pd.read_csv(path.replace("l20", length))
                ax.hist(df["0.0"], bins=bins, alpha=alpha, color="blue", label="Random")

                df = pd.read_csv(
                    path.replace("l20", length).replace("target", "source-target")
                )
                print_nav_and_abort(df, label="random")
                ax.vlines(
                    get_mean(df),
                    *ylim,
                    color="blue",
                    linestyles="--",
                    label="_nolegend_",
                )

                ax2 = ax.twinx()
                ax2.hlines(
                    get_aborted(df),
                    0,
                    1,
                    color="blue",
                    linestyles=":",
                    lw=DOT_LW,
                    # label="aborted\n(random)",
                    label="_nolegend_",
                )
                ax2.set_ylim(0, 1)

                df = pd.read_csv(
                    path.replace("l20", length).replace("random", shape_fitness)
                )
                ax.hist(
                    df["0.0"],
                    bins=bins,
                    alpha=alpha,
                    color="orange",
                    label=shape_fitness.title(),
                )

                df = pd.read_csv(
                    path.replace("l20", length)
                    .replace("random", shape_fitness)
                    .replace("target", "source-target")
                )
                print_nav_and_abort(df, label=shape_fitness)
                ax.vlines(
                    get_mean(df),
                    *ylim,
                    color="orange",
                    linestyles="--",
                    # label="_nolegend_",
                    label=r"Navigability, $\left<\psi^\mathrm{evo}\right>$",
                    zorder=-1,
                )

                ax2.hlines(
                    get_aborted(df, nav_threshold),
                    0,
                    1,
                    color="orange",
                    linestyles=":",
                    lw=DOT_LW,
                    zorder=-2,
                    # label="Aborted\n(hamming)",
                    label="Aborted, " r"$\alpha$",
                )

                if all_label:
                    if i == 0:
                        ax.set_ylabel(r"Count")
                    if i == len(axs[0]) - 1:
                        ax2.set_ylabel(r"Aborted, $\alpha$")
                else:
                    if i == 0 and row == len(axs)//2:
                        ax.set_ylabel(r"Count")
                    if i == len(axs[0]) - 1 and row == len(axs)//2:
                        ax2.set_ylabel(r"Aborted, $\alpha$")

                ax2.set_ylim(0, 1)

                ax.set_ylim(*ylim)

                if i != len(paths) - 1:
                    ax2.axes.get_yaxis().set_ticklabels([])
                if i == len(paths) - 1 and row == 1:
                    ax2.set_ylabel(r"Aborted, $\alpha$")

                if row == 0 and i == 0:
                    leg_axs = [ax, ax2]
                    ax00_x = axs[0, 0].get_position().bounds[0]
                    ax00_y = (
                        axs[0, 0].get_position().bounds[1]
                        + axs[0, 0].get_position().bounds[3]
                    )
                    lines_labels = [ax.get_legend_handles_labels() for ax in leg_axs]
                    lines, labels = [sum(lol, []) for lol in zip(*lines_labels)]
                    fig.legend(
                        lines,
                        labels,
                        loc="lower left",
                        prop={"size": "small"},
                        bbox_to_anchor=(ax00_x, ax00_y + 0.04),
                        ncol=2,
                        frameon=False,
                    )

        axs[0, 0].set_title(r"$N = 10$", fontsize=title_fs)
        axs[0, 1].set_title(r"$N = 1,000$", fontsize=title_fs)
        axs[0, 2].set_title(r"$N = 100,000$", fontsize=title_fs)
        axs[-1, 1].set_xlabel(
            r"Phenotypic evolutionary navigability, $\left<\psi_p^\mathrm{evo}\right>$"
        )

        axs[1, 0].set_ylabel(r"Count")

        for i, length in enumerate(lengths):
            axs[i, 0].text(
                -0.6,
                1.05,
                f"L={length[1:]}",
                transform=axs[i, 0].transAxes,
                ha="left",
                # fontsize="small",
                fontsize=nee_fs,
                weight="bold",
            )

        plt.subplots_adjust(hspace=0.3)

        plt.savefig(
            os.path.join(OUTS_FINAL, f"{fname}.pdf"),
            transparent=True,
            bbox_inches="tight",
        )
        plt.savefig(
            os.path.join(OUTS_FINAL, f"{fname}.png"),
            transparent=True,
            bbox_inches="tight",
            dpi=DPI,
        )
        plt.show()

In [ ]:
# Plot figs
figsize = (ONE_COL_MM / 250, ONE_COL_MM / 250 * ((3 * 1.2) / (3.45 * 1.25)))

# Aggregate figure for monomorphic population sizes: n=10, 1000, 100000
plot_poly_comparison(
    ns=[10, 1000, 100000],
    path=None,
    shape_fitness="hamming",
    fname="random_vs_hamming_poly",
    lengths=["l20", "l30", "l40"],
    fitness_type="most_common",
    ylim=(0, 50),
    all_label=False,
    figsize=figsize,
)

# Aggregate figure for monomorphic population sizes: n=10, 1000, 100000
plot_mono_comparison(
    ns=[10, 1000, 100000],
    path="20220213_outs_rna_mono_rdo",
    shape_fitness="hamming",
    fname="random_vs_hamming_mono",
    lengths=["l20", "l30", "l40"],
    ylim=(0, 50),
    all_label=False,
    figsize=figsize,
)


# Aggregate figure for monomorphic population sizes: n=10, 1000, 100000, shape level=3
plot_mono_comparison(
    ns=[10, 1000, 100000],
    path="20220327_outs_rna_shape_l3_mono_rdo",
    shape_fitness="lev",
    fname="random_vs_hamming_mono_shape_l3",
    lengths=["l60", "l100", "l140"],
    ylim=(0, 20),
    all_label=False,
    figsize=figsize,
)

# Aggregate figure for monomorphic population sizes: n=10, 1000, 100000, shape level=3
plot_mono_comparison(
    ns=[10, 1000, 100000],
    path="20220327_outs_rna_shape_mono_rdo",
    shape_fitness="lev",
    fname="random_vs_hamming_mono_shape_l1",
    lengths=["l60", "l100", "l100"],
    ylim=(0, 20),
    all_label=True,
    figsize=figsize,
)

In [ ]:
# Make similar figure comparing monomorphic with and without neutral mutations
# Aggregate figure:
# n, LEG, LABS = 1000, not True, not False


def plot_neut_mat_comparison(
    n=1000, LEG=True, LABS=True, figsize=(3.45, 3), COMP_ONLY=0
):
    paths = [
        f"{DATA_PATH}/data/raw_output/20220213_outs_rna_mono_rdo/random/l20/n{n}/analysis/fitness_type-mono/diff_thresh-0.00/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
        f"{DATA_PATH}/data/raw_output/20220225_outs_rna_mono_rdo_no-neut-mut/random/l20/n{n}/analysis/fitness_type-mono/diff_thresh-0.00/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
    ]
    with plt.rc_context(SMALL_RCPARAMS):
        # fig, axs = plt.subplots(3, 2, sharey=True, sharex=True, figsize=(4, 4))
        fig, axs = plt.subplots(3, 2, sharey=True, sharex=True, figsize=figsize)
        bins = np.arange(0, 1.01, 0.1)
        alpha = 0.5
        ylim = (0, 50)
        # title_fs = "medium"
        title_fs = nee_fs
        lengths = ["l20", "l30", "l40"]
        fitness_types = ["random", "hamming"]
        colors = {"random": "blue", "hamming": "orange"}

        for row, length in enumerate(lengths):
            for i, fitness_type in enumerate(fitness_types):
                print(f"L={length}, fitness={fitness_type}")
                ax = axs[row, i]
                df = pd.read_csv(
                    paths[0].replace("l20", length).replace("random", fitness_type)
                )
                ax.hist(
                    df["0.0"],
                    bins=bins,
                    alpha=alpha,
                    color=colors[fitness_type],
                    # label=fitness_type,
                    label="With neutral",
                )

                df = pd.read_csv(
                    paths[0]
                    .replace("l20", length)
                    .replace("random", fitness_type)
                    .replace("target", "source-target")
                )
                ax.vlines(
                    get_mean(df),
                    *ylim,
                    color=colors[fitness_type],
                    linestyles="--",
                    label="_nolegend_",
                )

                ax2 = ax.twinx()
                ax2.hlines(
                    get_aborted(df),
                    0,
                    1,
                    color=colors[fitness_type],
                    linestyles=":",
                    lw=DOT_LW,
                    label="aborted",
                )
                ax2.set_ylim(0, 1)
                print_nav_and_abort(df, label="neutral")
                df = pd.read_csv(
                    paths[1].replace("l20", length).replace("random", fitness_type)
                )
                color = (
                    f"light{colors[fitness_type]}"
                    if fitness_type == "random"
                    else "peachpuff"
                )
                ax.hist(
                    df["0.0"],
                    bins=bins,
                    alpha=alpha,
                    color=color,
                    # label=f"{fitness_type}\n(no neut muts)",
                    label=f"Without neutral",
                )

                df = pd.read_csv(
                    paths[1]
                    .replace("l20", length)
                    .replace("random", fitness_type)
                    .replace("target", "source-target")
                )
                ax.vlines(
                    get_mean(df),
                    *ylim,
                    color=color,
                    linestyles="--",
                    label="_nolegend_",
                )

                ax2.hlines(
                    get_aborted(df),
                    0,
                    1,
                    color=color,
                    linestyles=":",
                    lw=DOT_LW,
                    label="aborted\n(no neut muts)",
                )
                print_nav_and_abort(df, label="no neutral")
                ax2.set_ylim(0, 1)

                ax.set_ylim(*ylim)

                if i != len(paths) - 1:
                    ax2.axes.get_yaxis().set_ticklabels([])
                if i == len(paths) - 1 and row == 1:
                    ax2.set_ylabel(r"Aborted, $\alpha$")

        if LEG:
            axs[0, 0].legend(
                loc="lower left",
                bbox_to_anchor=(0.0, 1.32),
                prop={"size": "x-small"},
                ncol=1,
                frameon=False,
            )
            axs[0, 1].legend(
                loc="lower left",
                bbox_to_anchor=(0.0, 1.32),
                prop={"size": "x-small"},
                ncol=1,
                frameon=False,
            )

        axs[0, 0].set_title(f"Random", fontsize=title_fs)
        axs[0, 1].set_title(f"Hamming", fontsize=title_fs)

        pos2 = axs[-1, 1].get_position()
        right = pos2.bounds[0]
        pos1 = axs[-1, 0].get_position()
        left = pos1.bounds[0] + pos1.bounds[2]
        fig.text(
            (left + right) / 2,
            0.02,
            r"Phenotypic evolutionary navigability, $\left<\psi_p^\mathrm{evo}\right>$",
            ha="center",
        )
        axs[1, 0].set_ylabel(r"Count")

        if LABS:
            for i, length in enumerate(lengths):
                axs[0, 0].text(
                    # -0.5,
                    -0.3,
                    1.05,
                    f"L={length[-2:]}",
                    transform=axs[i, 0].transAxes,
                    ha="left",
                    # weight="bold",
                    weight="normal",
                )
                axs[0, 0].text(
                    # -0.5,
                    -0.3,
                    1.05,
                    f"L={length[-2:]}",
                    transform=axs[i, 0].transAxes,
                    ha="left",
                    # weight="bold",
                    weight="normal",
                )

        fname = f"neutral_mutations_vs_no_neutral_mutations_n{n:.0f}"

        plt.subplots_adjust(hspace=0.3)

        plt.savefig(
            os.path.join(OUTS_FINAL, f"{fname}.pdf"),
            transparent=True,
            bbox_inches="tight",
        )
        plt.savefig(
            os.path.join(OUTS_FINAL, f"{fname}.png"),
            transparent=True,
            bbox_inches="tight",
            dpi=DPI,
        )
        plt.show()

In [ ]:
# Plot neutral mutations comparison
figsize = (ONE_COL_MM/250, 3/3.45 * ONE_COL_MM / 250)
plot_neut_mat_comparison(n=10, LEG=True, LABS=True, figsize=figsize)
plot_neut_mat_comparison(n=1000, LEG=True, LABS=True, figsize=figsize)
plot_neut_mat_comparison(n=100000, LEG=True, LABS=True, figsize=figsize)

In [ ]:
def get_critical_threshold(df, critical_nav=0.5):
    """Get threshold at which navigability is over 50%."""
    try:
        row = df[df["nav_mean"] > critical_nav].drop_duplicates(keep="first").iloc[0]
        crit = row["threshold"]
        abort = row["aborted_mean"]
        nav = row["nav_mean"]
        return crit, nav, abort
    except:
        return np.nan, np.nan, 1.0


def get_critical_df(fitness_type="most_common", COMP_ONLY=1, critical_nav=0.5):
    """Get dataframe of critical thresholds and Aborteds."""

    # n=100
    paths = [
        (
            f"{DATA_PATH}/data/raw_output/20220213_outs_rna_mono_rdo/random/l20/n100/analysis/fitness_type-mono/diff_thresh-0.00/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
            "Monomorphic",
        ),
        (
            f"{DATA_PATH}/data/raw_output/20220220_outs_rna_mul_001/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
            "Intermediate",
        ),
        (
            f"{DATA_PATH}/data/raw_output/20220220_outs_rna_mul_1/random/l20/analysis/fitness_type-{fitness_type}/diff_thresh-0.50/comp_thresh-0.00/comp_only-{COMP_ONLY}/single_target.csv",
            "Polymorphic",
        ),
    ]

    bins = np.arange(0, 1.01, 0.1)
    alpha = 0.5
    ylim = (0, 20)
    # title_fs = "medium"
    title_fs = nee_fs
    lengths = ["l20", "l30", "l40"]
    outs = []
    for row, length in enumerate(lengths):
        for i, (path, label) in enumerate(paths):
            int_length = int(length[1:])
            load_path = path.replace("l20", length).replace("target", "source-target")
            df = pd.read_csv(load_path)

            crit, nav, aborted = get_critical_threshold(df, critical_nav=critical_nav)

            outs.append([int_length, label, fitness_type, "random", crit, nav, aborted])

            df = pd.read_csv(
                path.replace("l20", length)
                .replace("random", "hamming")
                .replace("target", "source-target")
            )
            crit, nav, aborted = get_critical_threshold(df, critical_nav=critical_nav)
            outs.append(
                [int_length, label, fitness_type, "hamming", crit, nav, aborted]
            )

    # Make `outs` dataframe
    outs = pd.DataFrame(
        outs,
        columns=[
            "Length",
            "Regime",
            "Fitness type",
            "Fitness function",
            "Critical threshold",
            "Navigability",
            "Aborted",
        ],
    )

    return outs


def get_critical_df_reorganised(outs, include_nav=False):
    """Reorganise critical and aborted df."""
    outs_unstack = (
        outs.drop(columns="Fitness type")
        .set_index(["Length", "Fitness function", "Regime"])
        .unstack()
    )

    idxs = [
        (20, "random"),
        (20, "hamming"),
        (30, "random"),
        (30, "hamming"),
        (40, "random"),
        (40, "hamming"),
    ]

    if include_nav:
        cols = [
            ("Critical threshold", "Monomorphic"),
            ("Critical threshold", "Intermediate"),
            ("Critical threshold", "Polymorphic"),
            ("Navigability", "Monomorphic"),
            ("Navigability", "Intermediate"),
            ("Navigability", "Polymorphic"),
            ("Aborted", "Monomorphic"),
            ("Aborted", "Intermediate"),
            ("Aborted", "Polymorphic"),
        ]
    else:
        cols = [
            ("Critical threshold", "Monomorphic"),
            ("Critical threshold", "Intermediate"),
            ("Critical threshold", "Polymorphic"),
            ("Aborted", "Monomorphic"),
            ("Aborted", "Intermediate"),
            ("Aborted", "Polymorphic"),
        ]

    outs_unstack = outs_unstack.loc[idxs, cols].round(3)
    display(outs_unstack)
    outs_unstack = outs_unstack.reorder_levels([1, 0], axis=1)

    return outs_unstack


def get_critical_df_reorganise_formatted(outs_untsack):
    """Format for paper."""
    fmt = lambda x: f"{x:.3f}" if not np.isnan(x) else " - "
    new_data = []
    for i in range(outs_unstack.shape[1] // 2):
        new_data.append(
            (
                outs_unstack.iloc[:, i].map(fmt)
                + " ("
                + outs_unstack.iloc[:, outs_unstack.shape[1] // 2 + i].map(fmt)
                + ")"
            )
        )

    final_outs = pd.concat(new_data, axis=1)
    final_outs.columns = [el[0] for el in outs_unstack.columns][
        : outs_unstack.shape[1] // 2
    ]
    final_outs = final_outs.reset_index()
    return final_outs

In [ ]:
# Make table for critical and aborted values
for include_nav in [False]:
    for comp_only in [0, 1]:
        outs = get_critical_df("most_common", COMP_ONLY=comp_only, critical_nav=0.5)
        outs_unstack = get_critical_df_reorganised(outs, include_nav=include_nav)
        final_outs = get_critical_df_reorganise_formatted(outs_unstack)

        if not include_nav:
            fname = f"critical_and_aborted_values_comp-only_{comp_only}"
            # markdown
            with open(os.path.join(OUTS_FINAL, f"{fname}.md"), "w") as f:
                final_outs.to_markdown(f, index=None)
            # tex table
            with open(os.path.join(OUTS_FINAL, f"{fname}.tex"), "w") as f:
                final_outs.to_latex(f, index=None)

In [ ]:
# Get comparison between critical and navigability at zero at comp_only = 0
conly = 0
display(
    # get_critical_df("most_common", COMP_ONLY=0, critical_nav=0.5).set_index(["Length", "Regime", "Fitness type", "Fitness function"])
    get_critical_df("most_common", COMP_ONLY=0, critical_nav=0.9)
    .set_index(["Length", "Regime", "Fitness type", "Fitness function"])
    .join(
        get_critical_df("most_common", COMP_ONLY=0, critical_nav=-1).set_index(
            ["Length", "Regime", "Fitness type", "Fitness function"]
        ),
        rsuffix="_0",
    )
    .drop(columns=["Critical threshold_0"])
    .round(3)
)

In [ ]:
# Get comparison between critical and navigability at zero at comp_only = 1
conly = 1
display(
    get_critical_df("most_common", COMP_ONLY=conly, critical_nav=0.5)
    .set_index(["Length", "Regime", "Fitness type", "Fitness function"])
    .join(
        get_critical_df("most_common", COMP_ONLY=conly, critical_nav=-1).set_index(
            ["Length", "Regime", "Fitness type", "Fitness function"]
        ),
        rsuffix="_0",
    )
    .drop(columns=["Critical threshold_0"])
    .round(3)
)

In [ ]:
# Get fRNA information
def Np(L, scaling=1.76):
    """Number of RNA phenotypes at length L."""
    return scaling ** L


for L in [60, 100, 140]:
    print(f"Length {L:3.0f}, N_p = {Np(L):.2e}")

# Calculate the number of structures and genotypes at each length
frnas = pd.read_csv(f"{DATA_PATH}/frna/data2/all_seqs_raw_all.csv", header=None)
frnas.columns = [
    "Genotypes",
    "dot-bracket",
    "Shape level 1",
    "Shape level 2",
    "Shape level 3",
    "Shape level 4",
    "Shape level 5",
]
frnas["L"] = frnas["Genotypes"].str.len()
Ls = [20, 30, 40, 60, 100, 140]
aggs = frnas.groupby("L")[
    ["Genotypes", "dot-bracket", "Shape level 1", "Shape level 3"]
].agg(pd.Series.nunique)
# aggs = aggs[(aggs.index >= 60) & (aggs.index <= 160)]
aggs = aggs[(aggs.index <= 300)]
# .loc[Ls]
N = 4
plt.rcParams["axes.prop_cycle"] = plt.cycler("color", plt.cm.viridis(np.linspace(0,1,N)))
fig, ax = plt.subplots(1, 1, figsize=(4, 3))
ax.plot(aggs.index, aggs, label=aggs.columns)
ax.legend(prop={"size":"small"})
ax.set(xlabel=r"Seqence length, $L$", ylabel="Unique counts in fRNAdb")
ax.set_yscale("log")
ax.vlines(30, 0, aggs.max().max(), ls=":", color="C1")
ax.vlines(80, 0, aggs.max().max(), ls=":", color="C2")
ax.vlines(120, 0, aggs.max().max(), ls=":", color="C3")
plt.savefig(os.path.join(OUTS_FINAL, "fRNA_counts.pdf"), bbox_inches="tight", transparent=True)

print(aggs.loc[[60, 80, 100, 120, 140, 160]].to_latex())

pd.set_option("display.max_colwidth", None)
print(frnas[frnas["L"] == 60].iloc[0].to_latex())